In [2]:
# Salaries in Science


#### Kayla Wen, Colin Chen, Paria Sokhanran, Justin Leung 

In [5]:
#Run this cell first

library(tidyverse)
library(testthat)
library(digest)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)


In [9]:
## reads the data of the GPA of Science Majors in UBC Undergrauates
gpa_data <- read_csv(file = "https://raw.githubusercontent.com/colin-c/Salaries-in-Science/main/data/ubc-science-gpa.csv")

gpa_data

## reads the data of the average salary of graduates with a certain degree and major
major_data <- read_csv(file = "https://raw.githubusercontent.com/colin-c/Salaries-in-Science/main/data/all-ages.csv")

major_data

Rows: 480 Columns: 9
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): Option, Spec, Space
dbl (6): Year, Intial Admit
Number, Inital Reject 
Number, Final Admit 
Numb...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Year,Option,Spec,Space,Intial Admit Number,Inital Reject Number,Final Admit Number,Max grade,Min Grade
<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2015,NA,NA,NA,NA,NA,NA,NA,NA
2015,CPSC,NA,193,NA,NA,NA,NA,NA
2015,CPSC,Major Cognitive Systems(1226): Computational Intelligence and Design,NA,9,15,10,87.58,70.33
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
2022,Combined Honours,"Combined Honours (0517): Chemical Physics (CHEM, PHYS)",NA,3,0,3,NA,NA
2022,Combined Honours,"Combined Honours (0009): Physics and Astronomy (PHYS, ASTR)",NA,11,0,10,90.19,74.62
2022,Combined Honours,"Combined Honours (0014): Physics and Mathematics (PHYS, MATH)",NA,13,0,13,96.30,77.90


Rows: 173 Columns: 12
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (2): Major, Major_category
dbl (10): index, Major_code, Total, Employed, Employed_full_time_year_round,...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


index,Major_code,Major,Major_category,Total,Employed,Employed_full_time_year_round,Unemployed,Unemployment_rate,Median,P25th,P75th
<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0,1100,GENERAL AGRICULTURE,Agriculture & Natural Resources,128148,90245,74078,2423,0.02614711,50000,34000,80000
1,1101,AGRICULTURE PRODUCTION AND MANAGEMENT,Agriculture & Natural Resources,95326,76865,64240,2266,0.02863606,54000,36000,80000
2,1102,AGRICULTURAL ECONOMICS,Agriculture & Natural Resources,33955,26321,22810,821,0.03024832,63000,40000,98000
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
170,6299,MISCELLANEOUS BUSINESS & MEDICAL ADMINISTRATION,Business,102753,77471,61603,4308,0.05267856,53000,36000,83000
171,6402,HISTORY,Humanities & Liberal Arts,712509,478416,354163,33725,0.06585101,50000,35000,80000
172,6403,UNITED STATES HISTORY,Humanities & Liberal Arts,17746,11887,8204,943,0.07349961,50000,39000,81000
